<a href="https://colab.research.google.com/github/LuisaUpegui/sales_predictions-/blob/main/sales_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import pandas as pd 
filename="/content/drive/MyDrive/Colab Notebooks/Coding dojo/sales-predictions/sales_predictions.csv"

In [131]:
df=pd.read_csv(filename)

In [89]:
 #¿Cuántas filas y columnas?  Hay 8523 filas y 12 columnas 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [62]:
#¿Cuáles son los tipos de datos en cada variable?
# Hay 4 columnas de tipo flotante, 1 columna con tipo de variable entero, y  7 columnas con tipo de variable categorica (object)

In [90]:
#3) ¿Hay duplicados? Si es el caso, eliminen algunos duplicados. No hay valores duplicados 
df.duplicated().sum

<bound method NDFrame._add_numeric_operations.<locals>.sum of 0       False
1       False
2       False
3       False
4       False
        ...  
8518    False
8519    False
8520    False
8521    False
8522    False
Length: 8523, dtype: bool>

In [91]:
#4) Identifiquen los valores faltantes. Hay dos columnas con valores faltantes, la columna item_weight 
# y la columna outlet_size 
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [92]:
#5) Decidan cómo abordar los valores faltantes y cómo hacerlo. (Esto requiere sus criterios,así que expliquen su elección).

In [93]:
df[df["Item_Identifier"]== "FDP10"]



,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
585,FDP10,19.0,Low Fat,0.128066,Snack Foods,104.3622,OUT035,2004,Small,Tier 2,Supermarket Type1,1905.5196
2623,FDP10,19.0,Low Fat,0.128815,Snack Foods,107.6622,OUT017,2007,NaN,Tier 2,Supermarket Type1,1164.4842
3382,FDP10,19.0,Low Fat,0.128289,Snack Foods,104.9622,OUT049,1999,Medium,Tier 1,Supermarket Type1,1164.4842
4585,FDP10,19.0,Low Fat,0.128090,Snack Foods,107.0622,OUT046,1997,Small,Tier 1,Supermarket Type1,1376.2086
6087,FDP10,19.0,Low Fat,0.128350,Snack Foods,106.5622,OUT045,2002,NaN,Tier 2,Supermarket Type1,1482.0708
7883,FDP10,19.0,Low Fat,0.127984,Snack Foods,107.6622,OUT013,1987,High,Tier 3,Supermarket Type1,1270.3464


In [137]:
#La columna "Item_Weighth" esta asociado con la columna "Item_Identifier", despues de hacer una busqueda se pudo encontrar
# que el item con  identificador FDP10 era el que presentaba valores faltantes pero algunas de sus filas estaban
#completamente llenas es por eso que para rellenar "Item_Weighth" se va a usar el valor puesto en el resto de filas el cual es 
# 19.0

df['Item_Weight'].fillna(19.0, inplace=True)



In [ ]:
#Para la columna "Outlet_Size" pasa algo similar al punto anterior pero esta vez hay una asociación entre la columna
# "Outlet_Size" y "Outlet_Type"== grosery store-->  según los datos puede inferirse que los datos faltantes 
# tienen valor a  'Small' ya que en esta solo hay estos dos valores (nan, 'Small')

In [113]:
grosery = df["Outlet_Type"]=='Grocery Store'
df1 = df.loc[grosery, :]

In [114]:
#Valores que toma 'Outlet_Size' filtrada por 'Grocery Store
df1['Outlet_Size'].unique()

array([nan, 'Small'], dtype=object)

In [138]:
#Reemplazar los NA por "Small"
df['Outlet_Size'].fillna('Small', inplace=True)


In [125]:
#6. Confirmar que no hay valores faltantes después de abordarlos
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [132]:
#7. Encuentren y arreglen alguna categoría inconsistente de datos 
#La columna llamada Item_Fat_Content tiene LF, Low Fat y low fat (estandarizar por Low Fat) y 
#lo mismo para el valor Regular y reg
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular', 'low fat', 'LF', 'reg'], dtype=object)

In [133]:
change={'low fat': 'Low Fat','Low Fat':'Low Fat', 'LF':'Low Fat',"reg":'Regular','Regular':'Regular' }

In [135]:
df['Item_Fat_Content']= df['Item_Fat_Content'].map(change)

In [136]:
df['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [49]:
 #8. Para cualquier columna numérica, obtengan las estadísticas resumidas para cada uno (mínimo, máximo y media)

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                8523 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                8523 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [51]:
df['Item_Weight'].describe()


count    8523.000000
mean       12.831222
std         4.445074
min         4.555000
25%         9.000000
50%        12.600000
75%        16.600000
max        21.350000
Name: Item_Weight, dtype: float64

In [52]:
df['Item_Visibility'].describe()


count    8523.000000
mean        0.066132
std         0.051598
min         0.000000
25%         0.026989
50%         0.053931
75%         0.094585
max         0.328391
Name: Item_Visibility, dtype: float64

In [ ]:
df['Item_MRP'].describe()


count    8523.000000
mean      140.992782
std        62.275067
min        31.290000
25%        93.826500
50%       143.012800
75%       185.643700
max       266.888400
Name: Item_MRP, dtype: float64

In [ ]:
df['Outlet_Establishment_Year'].describe()


count    8523.000000
mean     1997.831867
std         8.371760
min      1985.000000
25%      1987.000000
50%      1999.000000
75%      2004.000000
max      2009.000000
Name: Outlet_Establishment_Year, dtype: float64

In [ ]:
df['Item_Outlet_Sales'].describe()


count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64